# EP 2: Machine Learning 101

📚 สนใจเรียน/ปรึกษา(Fastwork): <a href="https://fastwork.co/user/krittachailouis/tutoring-12693427">https://fastwork.co/user/krittachailouis/tutoring-12693427</a>

<img width="32" alt="YouTube icon" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/32px-YouTube_full-color_icon_%282017%29.svg.png?20240107144800"> Youtube:
 <a href="https://commons.wikimedia.org/wiki/File:YouTube_full-color_icon_(2017).svg">
  Louis MakerLab
 </a>

<img width="32" alt="GitHub icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"> GitHub: 
<a href="https://github.com/louisa9555">
  Louis MakerLab
</a>


## Linear Models, Tree-based Models และ Kernel Methods
### เนื้อหาในบทเรียนนี้:
1. **Linear Models (โมเดลเชิงเส้น)**
   - Linear Regression (การถดถอยเชิงเส้น)
   - Logistic Regression (การถดถอยโลจิสติก)
2. **Tree-based Models (โมเดลที่ใช้ต้นไม้)**
   - Decision Trees (ต้นไม้ตัดสินใจ)
   - Random Forests (ป่าสุ่ม)
   - XGBoost
3. **Kernel Methods (วิธีเคอร์เนล)**
   - Support Vector Machine (SVM)
   - Gaussian Processes
---

### วัตถุประสงค์การเรียนรู้:
- เข้าใจหลักการทำงานของโมเดลแต่ละประเภท
- สามารถเลือกโมเดลที่เหมาะสมกับปัญหาที่กำหนด
- ใช้โค้ด Python เพื่อสร้างและประเมินโมเดล
- เปรียบเทียบประสิทธิภาพของโมเดลต่างๆ

## การติดตั้ง
- สร้าง conda env
- `conda install -c conda-forge llvm-openmp`
- `brew install libomp`
- `pip install numpy pandas matplotlib seaborn scikit-learn xgboost statsmodels`

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import joblib
from sklearn.datasets import make_classification, make_regression, load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
# Linear Models
from sklearn.linear_model import LinearRegression, LogisticRegression

# Tree-based Models
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb

# Kernel Methods
from sklearn.svm import SVC, SVR
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor

# การตั้งค่า matplotlib สำหรับการแสดงผลภาษาไทย
import matplotlib.font_manager as fm
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style("whitegrid")

print("นำเข้าไลบรารีเรียบร้อยแล้ว!")

# ฟังก์ชันสำหรับตั้งค่าฟอนต์ไทยใน matplotlib
def setup_matplotlib_thai_font():
    """Find and return a Thai font if available"""
    # Candidate font names that support Thai
    thai_fonts = ["TH Sarabun New", "Noto Sans Thai", "Noto Serif Thai", "Tahoma", "Angsana New"]

    # Get list of installed fonts
    font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
    for font_path in font_list:
        prop = fm.FontProperties(fname=font_path)
        try:
            name = prop.get_name()
            if any(tf in name for tf in thai_fonts):
                print(f"✔ Using Thai font: {name}")
                return prop
        except Exception:
            pass
    print("⚠️  ไม่พบฟอนต์ไทย, ใช้ฟอนต์เริ่มต้น.")
    return None

# ตั้งค่าฟอนต์ matplotlib
matplotlib_thai_font = setup_matplotlib_thai_font()

## ส่วนที่ 1: Linear Models (โมเดลเชิงเส้น)
<div style="display:flex; justify-content:center;">
  <img src="https://media.geeksforgeeks.org/wp-content/uploads/20231129130431/11111111.png" width="400">
</div>

<div style="display:flex; justify-content:center; margin-top:10px;">
  <img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/0*nKdtnOmCfBCgC7on.jpg" width="300">
</div>

### 1.1 Linear Regression (การถดถอยเชิงเส้น)

**ทฤษฎี:**
- Linear Regression เป็นอัลกอริทึมที่ใช้สำหรับปัญหาการทำนาย (regression)
- ค้นหาความสัมพันธ์เชิงเส้นระหว่างตัวแปรอิสระ (features) และตัวแปรตาม (target)
- สมการ: $y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n + \epsilon$
- **เป้าหมายหลัก** ของการฝึก Linear Regression คือการหา **slope และ Intercept** ที่ดีที่สุด

**จุดแข็ง:**
- เรียนรู้และคำนวณได้รวดเร็ว (ใช้ทรัพยากรน้อย)
- เข้าใจง่าย
- ไม่ต้องการการปรับพารามิเตอร์มาก

**จุดอ่อน:**
- ใช้ได้เฉพาะกับความสัมพันธ์เชิงเส้น(linear)เท่านั้น
- ไวต่อ outliers
- ต้องการสมมติฐานหลายประการ (linearity, independence, etc.)

Reference: https://www.freecodecamp.org/news/machine-learning-mean-squared-error-regression-line-c7dde9a26b93/ , 
https://www.geeksforgeeks.org/machine-learning/ml-linear-regression/

In [ ]:
# สร้างข้อมูลตัวอย่างสำหรับ Linear Regression
X_reg, y_reg = make_regression(n_samples=100, n_features=1, noise=20)
df = pd.DataFrame()
df['X'] = X_reg.flatten()
df['y'] = y_reg
print("ข้อมูลตัวอย่างสำหรับ Linear Regression:")
display(df.head(10))

# แบ่งข้อมูลเป็นชุดฝึก-ทดสอบ
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    df[['X']], df['y'], test_size=0.2
)

# สร้างโมเดล Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train_reg, y_train_reg)
# ทำนายผล
y_pred_reg = linear_reg.predict(X_test_reg)
print(y_pred_reg)
# คำนวณ Mean Squared Error
mse = mean_squared_error(y_test_reg, y_pred_reg)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Slope: {linear_reg.coef_[0]:.2f}")
print(f"Intercept: {linear_reg.intercept_:.2f}")
plt.show()

# แสดงกราฟ
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_train_reg, y_train_reg, alpha=0.7, label='ข้อมูล train')
plt.scatter(X_test_reg, y_test_reg, alpha=0.7, color='red', label='ข้อมูล test')
X_plot = np.linspace(df['X'].min(), df['X'].max(), 100).reshape(-1, 1)
df_X_plot = pd.DataFrame(X_plot, columns=['X'])
y_plot = linear_reg.predict(df_X_plot)
plt.plot(X_plot, y_plot, 'g-', linewidth=2, label='เส้น regression')
plt.xlabel('X', fontproperties=matplotlib_thai_font)
plt.ylabel('y', fontproperties=matplotlib_thai_font)
plt.title('Linear Regression', fontproperties=matplotlib_thai_font)
plt.legend(prop=matplotlib_thai_font)

plt.subplot(1, 2, 2)
plt.scatter(y_test_reg, y_pred_reg, alpha=0.7)
plt.plot([y_test_reg.min(), y_test_reg.max()], [y_test_reg.min(), y_test_reg.max()], 'r--', linewidth=2)
plt.xlabel('ค่าจริง', fontproperties=matplotlib_thai_font)
plt.ylabel('ค่าทำนาย', fontproperties=matplotlib_thai_font)
plt.title('ค่าจริง vs ค่าทำนาย', fontproperties=matplotlib_thai_font)
for i in range(len(y_test_reg)):
    plt.plot([y_test_reg.iloc[i], y_test_reg.iloc[i]], [y_test_reg.iloc[i], y_pred_reg[i]], 'k-', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# สร้างข้อมูลตัวอย่างแบบหลายตัวแปร
# 3 features: X1, X2, X3
X_reg, y_reg = make_regression(
    n_samples=150, n_features=3, noise=25
)
df = pd.DataFrame(X_reg, columns=['X1', 'X2', 'X3'])
df['y'] = y_reg
print("ข้อมูลตัวอย่างสำหรับ Multiple Linear Regression:")
display(df.head())

# แบ่งข้อมูลเป็น train-test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    df[['X1', 'X2', 'X3']], df['y'], test_size=0.2
)

# สร้างและฝึกโมเดล Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train_reg, y_train_reg)

# ทำนายผล
y_pred_reg = linear_reg.predict(X_test_reg)

# คำนวณ Mean Squared Error
mse = mean_squared_error(y_test_reg, y_pred_reg)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Intercept: {linear_reg.intercept_:.2f}")
print(f"Coefficients: {linear_reg.coef_}")

# แสดงกราฟค่าจริง vs ค่าทำนาย
plt.figure(figsize=(6, 6))
plt.scatter(y_test_reg, y_pred_reg, alpha=0.7, color='royalblue')
plt.plot([y_test_reg.min(), y_test_reg.max()], [y_test_reg.min(), y_test_reg.max()], 'r--', linewidth=2)
plt.xlabel('ค่าจริง', fontproperties=matplotlib_thai_font)
plt.ylabel('ค่าทำนาย', fontproperties=matplotlib_thai_font)
plt.title('Multiple Linear Regression\nค่าจริง vs ค่าทำนาย', fontproperties=matplotlib_thai_font)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Save model
joblib.dump(linear_reg, "models/linear_reg_model.pkl")
print("บันทึก Model สำเร็จ!")

# โหลด model
loaded_model = joblib.load("models/linear_reg_model.pkl")
# print(type(loaded_model))
# Predict
y_pred = loaded_model.predict(X_test_reg)

# Check model parameters
print("Model โหลดเรียบร้อยแล้ว!")
print("Coefficients:", loaded_model.coef_)
print("Intercept:", loaded_model.intercept_)
print("R² Score:", loaded_model.score(X_test_reg, y_test_reg))
display(pd.DataFrame({'Actual': y_test_reg, 'Predicted': y_pred}).head())

### 1.2 Logistic Regression (การถดถอยโลจิสติก)
Reference: https://www.geeksforgeeks.org/machine-learning/understanding-logistic-regression/
<div style="display:flex; justify-content:center;">
  <img src="https://upload.wikimedia.org/wikipedia/commons/c/cb/Exam_pass_logistic_curve.svg" width="400">
</div>

**ทฤษฎี:**
- Logistic Regression ใช้สำหรับปัญหาการจำแนกประเภท (classification)
- ใช้ sigmoid function เพื่อแปลงค่าให้อยู่ระหว่าง 0-1 (S-shape curve)
- สมการ: $p = \frac{1}{1 + e^{-(\beta_0 + \beta_1x_1 + ... + \beta_nx_n)}}$
- **เป้าหมายหลัก** ของการฝึก Logistic Regression คือการหา **ค่าน้ำหนัก (Weights หรือ $\beta$-coefficients)** ที่ดีที่สุด

เพื่อให้ค่าความน่าจะเป็นที่โมเดลทำนายได้ \(p\) ใกล้เคียงกับผลลัพธ์จริง (Actual Outcomes) มากที่สุด

**จุดแข็ง:**
- ให้ผลลัพธ์ในรูปของความน่าจะเป็น
- ไม่ต้องการสมมติฐานเกี่ยวกับการกระจายของข้อมูล
- ทนต่อ outliers ได้ดีกว่า Linear Regression

**จุดอ่อน:**
- ยังคงเป็นโมเดลเชิงเส้น
- อาจประสิทธิภาพไม่ดีกับข้อมูลที่ซับซ้อน

Data: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database?resource=download

Documents: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


| Solver              | Algorithm Type                | Best For            | Penalty Support     | Multi-class Support | Pros                                      | Cons                                     |
| ------------------- | ----------------------------- | ------------------- | ------------------- | ------------------- | ----------------------------------------- | ---------------------------------------- |
| **lbfgs**           | Quasi-Newton (L-BFGS)         | Medium datasets     | L2, none            | OvR, Multinomial    | Robust, default choice, accurate          | Higher memory use on very large datasets |
| **liblinear**       | Coordinate descent (like SVM) | Small/medium data   | L1, L2              | OvR only            | Works with sparse data, supports L1       | No multinomial, slower on large data     |
| **newton-cg**       | Newton’s method (conjugate)   | Medium/large data   | L2, none            | OvR, Multinomial    | Accurate for multi-class                  | Slower than lbfgs                        |
| **newton-cholesky** | Newton’s method (Cholesky)    | Medium/large data   | L2, none            | OvR, Multinomial    | Stable, can be faster than newton-cg      | Memory-intensive                         |
| **sag**             | Stochastic Average Gradient   | Large datasets      | L2, none            | OvR, Multinomial    | Very fast on large datasets               | Needs scaling, unstable on small data    |
| **saga**            | Stochastic Average Gradient + | Large/high-dim data | L1, L2, Elastic Net | OvR, Multinomial    | Most flexible (supports Elastic Net & L1) | Slower than lbfgs on small datasets      |


#### ทำความเข้าใจข้อมูล

In [ ]:
df = pd.read_csv('./data/diabetes.csv')
display(df.shape)

# หาค่าว่างของแต่ละคอลัมน์
empty = df.isnull().sum().any()
if empty:
    print("จำนวนแถวที่มีค่าว่างในแต่ละคอลัมน์:", df.isnull().sum())
else:
    print("ไม่พบค่าว่างในข้อมูล")

In [ ]:
plt.figure(figsize=(15, 20))
features = ['Pregnancies','Glucose','BloodPressure','SkinThickness',
            'Insulin','BMI','DiabetesPedigreeFunction','Age']
for i, col in enumerate(features, 1):
    plt.subplot(4, 2, i)
    sns.regplot(x=col, y='Outcome', data=df, logistic=True, ci=None, scatter_kws={'alpha':0.3})
    plt.title(f'Logistic Relationship: {col} vs Outcome')

plt.tight_layout()
plt.show()


In [ ]:
# สนใจ max_iter, penalty ['l1', 'l2', 'elasticnet'], 
# solver ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], n_job (-1 หมายถึงใช้ทุก processor)

# --- Prepare data ---
X = df.iloc[:, 0:8]
y = df.iloc[:, 8]
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)
print("X train shape:", X_train.shape)
print("X test shape:", X_test.shape)
print("y train shape:", y_train.shape)
print("y test shape:", y_test.shape)

# --- Compare penalties ---
models = {
    'L1': LogisticRegression(max_iter=500, penalty='l1', solver='liblinear'),
    'L2': LogisticRegression(max_iter=500, penalty='l2', solver='saga'),
    'ElasticNet': LogisticRegression(max_iter=500, penalty='elasticnet', solver='saga', l1_ratio=0.8)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(f"{name} model accuracy: {model.score(X_test, y_test):.3f}")

In [ ]:
features = ['Glucose','BMI','Age','Pregnancies']
# --- Prepare data ---
X = df[features]
y = df['Outcome']
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
print("X train shape:", X_train.shape)
print("X test shape:", X_test.shape)
print("y train shape:", y_train.shape)
print("y test shape:", y_test.shape)

# --- Compare penalties ---
models = {
    'L1': LogisticRegression(max_iter=500, penalty='l1', solver='liblinear'),
    'L2': LogisticRegression(max_iter=500, penalty='l2', solver='saga'),
    'ElasticNet': LogisticRegression(max_iter=500, penalty='elasticnet', solver='saga', l1_ratio=0.8)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(f"{name} model accuracy: {model.score(X_test, y_test):.3f}")

In [ ]:
# ดูค่า coefficients
for model in models.values():
    coef_df = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': model.coef_[0]
    }).sort_values('Coefficient', ascending=False)
    print(coef_df)
    break


In [ ]:
# Test โมเดล
for name, model in models.items():
    print(f'=== {name} model ===')
    # Predict
    y_pred = model.predict(X_test)

    # Accuracy and reports
    acc = model.score(X_test, y_test)
    print(f'Accuracy: {acc:.3f}')
    cm = confusion_matrix(y_test, y_pred)
    # print('Confusion Matrix:', cm)
    disp = ConfusionMatrixDisplay(cm, display_labels=getattr(model, 'classes_', None))
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {name} Model')
    plt.grid(False)
    plt.show()
    break

In [ ]:
# Save model
joblib.dump(models['L1'], "models/logistic_reg_model.pkl")
print("บันทึก Model สำเร็จ!")

# Load model
loaded_model = joblib.load("models/logistic_reg_model.pkl")
print("Model โหลดเรียบร้อยแล้ว!")
# Predict
y_pred = loaded_model.predict(X_test)
print("Coefficients:", loaded_model.coef_)
print("Intercept:", loaded_model.intercept_)
print("R² Score:", loaded_model.score(X_test, y_test))
display(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}).head())
